In [1]:
import tensorflow as tf
import numpy as np
def printl(a):
    print(a)
    print('====================================================')

# 数据类型
## 数值类型
### 标量,单个数字，shape为[]

In [54]:
a = 1.1
aa =tf.constant(a)
printl(type(a))
printl(type(aa))
printl(tf.is_tensor(aa))

<class 'float'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
True


### 向量shape 为[n,]
与标量不同，向量的定义须通过 List 类型传给 tf.constant()。

In [55]:
# 注意shape的区别，类似numpy
x = tf.constant([[1,2.,3.3]])
printl(x)
y = tf.constant([1,2.,3.3])
printl(y)


tf.Tensor([[1.  2.  3.3]], shape=(1, 3), dtype=float32)
tf.Tensor([1.  2.  3.3], shape=(3,), dtype=float32)


### 矩阵  shape为[n,m]
numpy()方法可以返回 Numpy.array 类型的数据，方便导出数据到系统的其他 模块：

In [56]:
x.numpy()

array([[1. , 2. , 3.3]], dtype=float32)

In [57]:
a = tf.constant([[1,2],[3,4]])
printl(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


### 3维张量（2纬以上）：  
第一维一般理解为"个数"

In [58]:
a = tf.constant([[[1,2],[3,4]],[[1,2],[3,4]],[[5,6],[7,8]]])
printl(a)

tf.Tensor(
[[[1 2]
  [3 4]]

 [[1 2]
  [3 4]]

 [[5 6]
  [7 8]]], shape=(3, 2, 2), dtype=int32)


## 字符串类型

In [59]:
a = tf.constant('Hello, Deep Learning.')
print(a)

tf.Tensor(b'Hello, Deep Learning.', shape=(), dtype=string)


在 tf.strings 模块中，提供了常见的字符串型的工具函数，如拼接 join()，长度 length()，切 分 split()等等：

In [60]:
b =tf.strings.lower(a)
print(b)

tf.Tensor(b'hello, deep learning.', shape=(), dtype=string)


## boolean 类型


In [61]:
a = tf.constant(True)
print(a)

tf.Tensor(True, shape=(), dtype=bool)


TensorFlow 的布尔类型和 Python 语言的布尔类型对等，能通用：

In [62]:
a = tf.constant(True)
if a:
    print('done')

done


# 数值精度
对于数值类型的张量，可以保持为不同字节长度的精度，如浮点数 3.14 既可以保存为 16-bit 长度，也可以保存为 32-bit 甚至 64-bit 的精度。Bit 位越长，精度越高，同时占用的 内存空间也就越大。常用的精度类型有 tf.int16, tf.int32, tf.int64, tf.float16, tf.float32, tf.float64，其中 tf.float64 即为 tf.double。  
不指定的话，**默认 `tf.float32` 和 `tf.int32`**  
对于大部分深度学习算法，一般使用 tf.int32, tf.float32 可满足运算精度要求，部分对 精度要求较高的算法，如**_强化学习_**，可以选择使用 `tf.int64`, `tf.float64` 精度保存张量。

In [113]:
a = tf.constant(1,dtype = tf.int16)
printl(a)
b =tf.constant(1.2)
printl(b)
c =tf.constant(1)
printl(c)

tf.Tensor(1, shape=(), dtype=int16)
tf.Tensor(1.2, shape=(), dtype=float32)
tf.Tensor(1, shape=(), dtype=int32)


## 精度转换
使用`tf.cast(data，type)`  

In [64]:
a = tf.constant(2 ,dtype = tf.int16)
a = tf.cast(a,tf.float32)
print(a)

tf.Tensor(2.0, shape=(), dtype=float32)


进行类型转换时，需要保证转换操作的合法性，例如将高精度的张量转换为低精度的张量时，可能发生数据**溢出隐患**：

In [65]:
a = tf.constant(123456789, dtype=tf.int32)

print(tf.cast(a, tf.int16))

tf.Tensor(-13035, shape=(), dtype=int16)


布尔型与整形之间相互转换  
整数转bool，只要不是0，都算True

In [66]:
a = tf.constant([True, False])
printl(tf.cast(a, tf.int16))
b = tf.constant([1, 2, 0, 4])
printl(tf.cast(b, tf.bool))


tf.Tensor([1 0], shape=(2,), dtype=int16)
tf.Tensor([ True  True False  True], shape=(4,), dtype=bool)


# 待优化张量
为了区分需要计算梯度信息的张量与不需要计算梯度信息的张量，TensorFlow 增加了 一种专门的数据类型来支持梯度信息的记录：`tf.Variable`。`tf.Variable` 类型在普通的张量类 型基础上添加了 `name`，`trainable` 等属性来支持计算图的构建。由于梯度运算会消耗大量的 计算资源，而且会自动更新相关参数，对于不需要的优化的张量，如神经网络的输入 `X`， 不需要通过 `tf.Variable` 封装；相反，对于需要计算梯度并优化的张量，如神经网络层的`W` 和`𝒃`，需要通过 `tf.Variable` 包裹以便 TensorFlow 跟踪相关梯度信息。

In [67]:
a = tf.constant([-1, 0, 1, 2])

aa = tf.Variable(a)

aa.name, aa.trainable

('Variable:0', True)

name 属性用于命名计算图中的变量，这套命名体系是 TensorFlow 内部维护的，一般**不需要用户关注 name 属性**；trainable 表征当前张量是否需要被优化，创建 Variable 对象是默认启用优化标志，可以设置 `trainable=False` 来设置张量不需要优化。

In [71]:
# 直接创建优化tensor，即 trainable的tensor
b = tf.Variable([2,3,4.])
b

<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([2., 3., 4.], dtype=float32)>

待优化张量可看做普通张量的特殊类型，普通张量也可以通过 `GradientTape.watch()`方法**<font color=red>临时</font>**加入跟踪梯度信息的列表。

# 创建张量
## 从 Numpy, List 对象创建
### `tf.convert_to_tensor()` 
可以创建新 Tensor，并将保存在 Python List 对象或者 Numpy Array 对象中的数据导入到新 Tensor 中：
需要注意的是，Numpy中浮点数数组默认使用 **64-Bit** 精度保存数据，转换到 Tensor 类型时精度为 **tf.float64**，可以在需要的时候转换为 tf.float32 类型。

In [72]:
tf.convert_to_tensor([1,2.])

<tf.Tensor: id=133, shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>

In [75]:
tf.convert_to_tensor(np.array([[1,2.],[3,4]]))

<tf.Tensor: id=134, shape=(2, 2), dtype=float64, numpy=
array([[1., 2.],
       [3., 4.]])>

## 创建全 0，全 1 张量
通过 `tf.zeros()`和 `tf.ones()`即可创建任意形 状全 0 或全 1 的张量

In [82]:
a = tf.zeros((3,4))
a

<tf.Tensor: id=155, shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

In [83]:
b = tf.ones((3,4))
b

<tf.Tensor: id=158, shape=(3, 4), dtype=float32, numpy=
array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)>

通过 `tf.zeros_like()`, `tf.ones_like()` 可以方便地新建与某个张量 shape 一致，内容全 0 或全 1 的张量。

In [85]:
a = tf.constant([[1,2,3],[3,4,5]])
b =tf.ones_like(a)
b

<tf.Tensor: id=166, shape=(2, 3), dtype=int32, numpy=
array([[1, 1, 1],
       [1, 1, 1]], dtype=int32)>

## 创建自定义数值张量
`tf.fill(shape, value)` shape必须是个一个向量，不能是矩阵，或者已有的tensor

In [103]:
a = tf.fill((3,4),99)
a

<tf.Tensor: id=189, shape=(3, 4), dtype=int32, numpy=
array([[99, 99, 99, 99],
       [99, 99, 99, 99],
       [99, 99, 99, 99]], dtype=int32)>

## 创建已知分布的张量
###  `tf.random.normal(shape, mean=0.0, stddev=1.0)`
可以创建形状为 shape，均值为 mean，标准差为 stddev 的正态分布𝒩(𝑚𝑒𝑎𝑛, 𝑠𝑡𝑑𝑑𝑒𝑣 2 )。默认 $mean= 0$ and $stddev = 1$

In [105]:
tf.random.normal([1,4])

<tf.Tensor: id=201, shape=(1, 4), dtype=float32, numpy=
array([[-0.8434031 , -1.3420401 ,  0.44487366, -0.4197785 ]],
      dtype=float32)>

In [106]:
tf.random.normal([1,4],20,10)

<tf.Tensor: id=207, shape=(1, 4), dtype=float32, numpy=array([[26.382725 , 19.394056 ,  2.7527008, 16.244581 ]], dtype=float32)>

### `tf.random.uniform(shape, minval=0, maxval=None, dtype=tf.float32)`
可以创建采样自 [𝑚𝑖𝑛𝑣𝑎𝑙, 𝑚𝑎𝑥𝑣𝑎𝑙]区间的均匀分布的张量。默认$minval = 0$ and $maxval = 1$  
如果需要均匀采样整形类型的数据，必须指定采样区间的最大值 maxval 参数，同时制定数 据类型为 `tf.int*`型：

In [115]:
tf.random.uniform([3,4],5,20,dtype = tf.int32)

<tf.Tensor: id=221, shape=(3, 4), dtype=int32, numpy=
array([[13, 15, 19, 18],
       [18, 18, 13,  6],
       [ 9, 16,  9, 12]], dtype=int32)>

## 创建序列
`tf.range()函数实现。tf.range(limit, delta=1)`可以创建[0, 𝑙𝑖𝑚𝑖𝑡)之间，步长为 delta 的整形序 列，不包含 limit 本身


In [116]:
tf.range(20)

<tf.Tensor: id=225, shape=(20,), dtype=int32, numpy=
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19], dtype=int32)>

In [124]:
tf.range(21,delta=3)

<tf.Tensor: id=262, shape=(7,), dtype=int32, numpy=array([ 0,  3,  6,  9, 12, 15, 18], dtype=int32)>

In [125]:
tf.range(10,21,delta=3)

<tf.Tensor: id=266, shape=(4,), dtype=int32, numpy=array([10, 13, 16, 19], dtype=int32)>

# 张量的典型应用
## 标量
`tf.reduce_mean()` 计算平均值

In [129]:
out = tf.random.uniform([4,10]) #随机模拟网络输出
print(out)
y = tf.constant([2,3,2,0]) # 随机构造样本真实标签

y = tf.one_hot(y, depth=10) # one-hot 编码
print(y)
loss = tf.keras.losses.mse(y, out) # 计算每个样本的 MSE
print(loss)
loss = tf.reduce_mean(loss) # 平均 MSE
print(loss)

tf.Tensor(
[[2.5363755e-01 1.6698968e-01 1.0132611e-01 3.2156944e-02 6.9034457e-01
  2.9455686e-01 7.8874445e-01 9.7741604e-01 4.6050549e-02 4.0034389e-01]
 [2.7263474e-01 2.6343882e-01 6.4742112e-01 5.9869289e-03 1.3470149e-01
  1.4554381e-01 5.1666629e-01 4.8307872e-01 8.7118387e-01 3.0769384e-01]
 [4.3996096e-01 7.6022303e-01 3.0946457e-01 4.3136072e-01 3.1962788e-01
  9.4238496e-01 8.4017599e-01 1.6197681e-01 8.5438359e-01 9.5582044e-01]
 [1.6968310e-01 3.1218374e-01 4.2423487e-02 1.8907833e-01 6.5188420e-01
  1.7191887e-02 1.9816482e-01 4.8855257e-01 4.9607027e-01 6.7353249e-04]], shape=(4, 10), dtype=float32)
tf.Tensor(
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(4, 10), dtype=float32)
tf.Tensor([0.32040614 0.2944219  0.48003635 0.17737228], shape=(4,), dtype=float32)
tf.Tensor(0.31805915, shape=(), dtype=float32)


## 向量
通过高层接口类 `Dense()`方式创建的网络层，张量 W 和𝒃存储在类的内部，由类自动创 建并管理。可以通过全连接层的 `bias` 成员变量查看偏置变量𝒃，例如创建输入节点数为 4， 输出节点数为 3 的线性层网络，那么它的偏置向量 b 的长度应为 3：

In [3]:
fc = tf.keras.layers.Dense(3) # 创建一层 Wx+b，输出节点为 3

# 通过 build 函数创建 W,b 张量，输入节点为 4

fc.build(input_shape=(2,4))

fc.bias # 查看偏置

<tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>

## 矩阵

In [6]:
fc = tf.keras.layers.Dense(3) # 定义全连接层的输出节点为 3

fc.build(input_shape=(2,4)) # 定义全连接层的输入节点为 4

fc.kernel

<tf.Variable 'kernel:0' shape=(4, 3) dtype=float32, numpy=
array([[-0.7828743 ,  0.37297952, -0.14484924],
       [ 0.8807198 , -0.41590613, -0.8967582 ],
       [ 0.16176462, -0.4971108 , -0.83295697],
       [ 0.29150033, -0.31457525,  0.22527957]], dtype=float32)>

## 3维张量
$$𝑋 = [𝑏, 𝑠𝑒𝑞𝑢𝑒𝑛𝑐𝑒 𝑙𝑒𝑛, 𝑓𝑒𝑎𝑡𝑢𝑟𝑒 𝑙𝑒𝑛]$$
其中𝑏表示序列信号的数量，sequence len 表示序列信号在时间维度上的采样点数，feature len 表示每个点的特征长度

In [9]:
# 自动加载 IMDB 电影评价数据集

(x_train,y_train),(x_test,y_test)=tf.keras.datasets.imdb.load_data(num_words=10000)

# 将句子填充、截断为等长 80 个单词的句子

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,maxlen=80)

x_train.shape

17465344/17464789 [==============================] - 3s 0us/step


(25000, 80)

可以看到 x_train 张量的 shape 为[25000,80]，其中 25000 表示句子个数，80 表示每个句子 共 80 个单词，每个单词使用数字编码方式。我们通过 `layers.Embedding()` 层将数字编码的单 词转换为长度为 100 个词向量：

In [111]:
# 创建词向量 Embedding 层类

embedding=tf.keras.layers.Embedding(10000, 100)

# 将数字编码的单词转换为词向量

out = embedding(x_train)

out.shape

TensorShape([25000, 80, 100])

## 4维张量
4 维张量在卷积神经网络中应用的非常广泛，它用于保存特征图(Feature maps)数据， 格式一般定义为

$$[𝑏, ℎ, w, 𝑐]$$
其中𝑏表示输入的数量，h/w分布表示特征图的高宽，𝑐 表示特征图的通道数，部分深度学 习框架也会使用[𝑏, 𝑐, ℎ, ]格式的特征图张量，例如 PyTorch。

In [17]:
# 创建 32x32 的彩色图片输入，个数为 4

x = tf.random.normal([4,32,32,3])

# 创建卷积神经网络

layer = tf.keras.layers.Conv2D(16,kernel_size=3)

out = layer(x) # 前向计算

out.shape # 输出大小


TensorShape([4, 30, 30, 16])

In [18]:
layer.kernel.shape

TensorShape([3, 3, 3, 16])

# 索引与切片
索引和numpy一样，可以A[a][b][c] 也可以A[a,b,c]  
切片也和numpy一样A[:,3,3] A[...,c]


# 维度变换
## reshape()
为了能够正确恢复出数据，必须保证张量的存储顺序与新视图的维度顺序一致，例如根据图片数量-行-列-通道初始视图保存的张量，按照图片数量-行-列-通道$(𝑏 − ℎ − w − 𝑐)$ 的顺序可以获得合法数据。如果按着图片数量-像素-通道$( b − h ∗ w − c)$的方式恢复视图，也能得到合法的数据。但是如果按着图片数量-通道-像素$(b − c − h ∗ w)$的方式恢复数据， 由于内存布局是按着图片数量-行-列-通道的顺序，视图维度与存储维度顺序相悖，提取的数据将是错乱的。  
举个例子，对 于 shape 为$[4,32,32,3]$的图片数据，通过 Reshape 操作将 shape 调整为$[4,1024,3]$，此时视图 的维度顺序为$𝑏 − 𝑝𝑖𝑥𝑒𝑙 − 𝑐$ ，张量的存储顺序为$[𝑏, ℎ, w, 𝑐]。$  
通过 `tf.reshape(x, new_shape)`，可以将张量的视图任意的合法改变

In [22]:
x = tf.range(96)
x = tf.reshape(x,[2,4,4,3])
x.ndim,x.shape


(4, TensorShape([2, 4, 4, 3]))

In [23]:
tf.reshape(x,[2,-1])

<tf.Tensor: id=208, shape=(2, 48), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
        32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47],
       [48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,
        64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
        80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95]],
      dtype=int32)>

其中的参数-1 表示当前轴上长度需要根据视图总元素不变的法则自动推导，从而方便用户书写。比如，上面的-1 可以推导为
$$ \frac{2*4*4*3}{2} = 48$$

## 增删维度
### `tf.expand_dims(x, axis)`
通过 `tf.expand_dims(x, axis)`可在指定的 **axis 轴前**可以插入一个新的维度  
tf.expand_dims 的 axis 为正时，表示在当前维度之前插入一个新维度  
为负时，表示**当前维度之后**插入一个新的维度  
**其实就是往靠近0的方向插入，无论正负**

In [48]:
a = tf.random.uniform((28,28),5,20)
a = tf.expand_dims(a,0)

In [49]:
# -1就是最后一个纬度，再最后一个纬度后面再插一个，就是最后加一个
a = tf.expand_dims(a,axis=-1)

### `tf.squeeze(x, axis)`
如果希望将 图片数量维度删除，可以通过 `tf.squeeze(x, axis)`函数，`axis` 参数为待删除的维度的索引号  
如果不指定维度参数 axis，即` tf.squeeze(x)`，那么他会默认**删除所有**长度为 1 的维度

In [50]:
a = tf.squeeze(a, axis=0)

In [51]:
a = tf.squeeze(a, axis=2)

## 交换维度
### `tf.transpose(x, perm) `
交换维度(Transpose)。通过交换维度，**改变了张量的存储顺序**，同时也改变了张量的视图。后续的所有操作必须**基于新的存续顺序**进行 。
原维度索引分别为 0,1,2,3
新纬度索引 0,3,1,2

In [53]:
a = tf.random.normal([2,32,32,3])
a = tf.transpose(a,perm=[0,3,1,2])

## 数据复制
### `tf.tile(x, multiples)`
multiples 分别指定了每个维度上面的复制倍数，对应位置为 1 表明不复制，为 2 表明新长度为原来的长度的 2 倍，即数据复制一份，以此类推。  
需要注意的是，tf.tile 会创建一个新的张量来保存复制后的张量，由于复制操作涉及到大量数据的读写 IO 运算，**计算代价相对较高**。

In [64]:
b = tf.constant([1,2])
print(b)
b = tf.expand_dims(b, axis=0)
print(b)

tf.Tensor([1 2], shape=(2,), dtype=int32)
tf.Tensor([[1 2]], shape=(1, 2), dtype=int32)


In [65]:
b = tf.tile(b, multiples=[3,1])
print(b)

tf.Tensor(
[[1 2]
 [1 2]
 [1 2]], shape=(3, 2), dtype=int32)


## Broadcasting
* 轻量级张量复制的手段  
* 只有在需要时才会执行实际存储复制操作  
**1. 先把纬度全部向右靠齐**  
**2. 广播后的纬度右侧部分一定要与原数据相同（原数据中的1可不相等）**

In [4]:
A = tf.random.normal([3,3])
tf.broadcast_to(A, [2,3,3,3])

<tf.Tensor: id=23, shape=(2, 3, 3, 3), dtype=float32, numpy=
array([[[[-0.3009113 , -0.47667378, -0.33907276],
         [ 0.90679324, -0.47989577,  0.5545086 ],
         [-0.3107798 ,  0.6111532 ,  2.9439762 ]],

        [[-0.3009113 , -0.47667378, -0.33907276],
         [ 0.90679324, -0.47989577,  0.5545086 ],
         [-0.3107798 ,  0.6111532 ,  2.9439762 ]],

        [[-0.3009113 , -0.47667378, -0.33907276],
         [ 0.90679324, -0.47989577,  0.5545086 ],
         [-0.3107798 ,  0.6111532 ,  2.9439762 ]]],


       [[[-0.3009113 , -0.47667378, -0.33907276],
         [ 0.90679324, -0.47989577,  0.5545086 ],
         [-0.3107798 ,  0.6111532 ,  2.9439762 ]],

        [[-0.3009113 , -0.47667378, -0.33907276],
         [ 0.90679324, -0.47989577,  0.5545086 ],
         [-0.3107798 ,  0.6111532 ,  2.9439762 ]],

        [[-0.3009113 , -0.47667378, -0.33907276],
         [ 0.90679324, -0.47989577,  0.5545086 ],
         [-0.3107798 ,  0.6111532 ,  2.9439762 ]]]], dtype=float32)>

在进行张量运算时，有些运算可以在处理不同 shape 的张量时，会隐式自动调用 Broadcasting 机制，如$+，-，*，/$ 等运算等，将参与运算的张量 **Broadcasting 成一个公共 shape**，再进行相应的计算

In [96]:
a = tf.random.normal([2,3,3,1])

b = tf.random.normal([3,3])
a+b

<tf.Tensor: id=632, shape=(2, 3, 3, 3), dtype=float32, numpy=
array([[[[-0.80892646,  0.91634285, -0.30629864],
         [-0.27354348,  1.5865757 , -0.98530245],
         [ 1.9690009 , -0.17135239,  0.15626061]],

        [[-0.46897167,  1.2562977 ,  0.03365618],
         [ 0.7185725 ,  2.5786917 ,  0.00681356],
         [ 0.25636768, -1.8839856 , -1.5563726 ]],

        [[-2.233618  , -0.50834864, -1.7309902 ],
         [ 0.09579784,  1.955917  , -0.61596113],
         [-1.4984066 , -3.63876   , -3.311147  ]]],


       [[[-0.84298015,  0.8822892 , -0.34035233],
         [ 1.3139806 ,  3.1740997 ,  0.6022216 ],
         [ 1.0649967 , -1.0753566 , -0.7477436 ]],

        [[-1.4116297 ,  0.31363973, -0.9090018 ],
         [ 0.08624664,  1.9463658 , -0.62551236],
         [ 0.94696456, -1.1933887 , -0.86577576]],

        [[-3.6648064 , -1.939537  , -3.1621785 ],
         [ 0.83670235,  2.6968215 ,  0.12494341],
         [ 1.3181095 , -0.8222438 , -0.49463084]]]], dtype=float32)>

# 数学运算
基本上和python3里的一样  

### 常用的 `tf.square(x)`, `tf.sqrt(x)`,`tf.exp(x)`
### 奇葩的`tf.math.log(x)`
如果希望计算其他底数的对数，可以根据对数的换底公式

In [97]:
x = tf.constant([1.,2.])
x = 10**x
tf.math.log(x)/tf.math.log(10.)

<tf.Tensor: id=639, shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>

### `tf.matmul(a, b)`
TensorFlow 中的矩阵 相乘可以使用批量方式，也就是张量 a,b 的维度数可以大于 2。当张量 a,b 维度数大于 2 时，TensorFlow 会选择 a,b 的最后两个维度进行矩阵相乘，前面所有的维度都视作 Batch 维度。

In [110]:
a = tf.random.normal([4,2,23,32])
b = tf.random.normal([4,2,32,2])
c = a@b
print(c.shape)

(4, 2, 23, 2)


矩阵相乘函数支持自动 Broadcasting 机制，但最后两位还是要符合矩阵乘法的

In [109]:
a = tf.random.normal([4,28,32])

b = tf.random.normal([32,16])
c = a@b
print(c.shape)

(4, 28, 16)
